In [10]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import paddle
def synthetic_data(w, b, num_examples):
    """生成y=Xw+b+噪声"""
    X = paddle.normal(0, 1, (num_examples, len(w)))
    y = paddle.matmul(X, w) + b
    y += paddle.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = paddle.to_tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print(len(true_w))

2


In [11]:

def load_array(data_arrays, batch_size, is_train=True):
    """构造一个Paddle数据迭代器"""
    dataset = paddle.io.TensorDataset(data_arrays)
    return paddle.io.DataLoader(dataset, batch_size=batch_size,
                                shuffle=is_train,
                                return_list=True)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[Tensor(shape=[10, 2], dtype=float32, place=Place(gpu:0), stop_gradient=True,
        [[-0.21706608, -0.05333515],
         [-1.15808201, -0.00936312],
         [ 0.64921069, -1.29995191],
         [ 1.18909514, -0.51112437],
         [-0.52607369,  0.23117609],
         [-0.50837523, -0.37801442],
         [ 2.02070999, -0.59552199],
         [-0.78526264, -2.53457332],
         [-0.60161507, -0.75507200],
         [ 1.21321237,  0.71976131]]),
 Tensor(shape=[10, 1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
        [[3.92788410 ],
         [1.90915704 ],
         [9.89881134 ],
         [8.29380512 ],
         [2.36448121 ],
         [4.47440910 ],
         [10.27638340],
         [11.24937153],
         [5.57783747 ],
         [4.18148613 ]])]

In [13]:
# nn是神经网络的缩写
from paddle import nn

net = nn.Sequential(nn.Linear(2, 1))

In [15]:
wei_attr = paddle.ParamAttr(initializer=paddle.nn.initializer.Normal(0, 0.01))
bias_attr = paddle.ParamAttr(initializer=None)
net = nn.Sequential(nn.Linear(2, 1, weight_attr=wei_attr, bias_attr=bias_attr))

In [17]:
loss = nn.MSELoss()

In [19]:
triner = paddle.optimizer.SGD(learning_rate=0.02, parameters=net.parameters())

In [25]:
num_epochs = 6
for epoch in range(num_epochs):
    for i,(X, y) in enumerate (data_iter()):
        l = loss(net(X) ,y)
        triner.clear_grad()
        l.backward()
        triner.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1},'f'loss {l}')

epoch 1,loss Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       0.00009634)
epoch 2,loss Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       0.00009663)
epoch 3,loss Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       0.00009567)
epoch 4,loss Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       0.00009546)
epoch 5,loss Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       0.00009569)
epoch 6,loss Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       0.00009536)
epoch 7,loss Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       0.00009563)
epoch 8,loss Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       0.00009652)
epoch 9,loss Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       0.00009617)
epoch 10,loss Tensor(shape=[], dtype=float32, place=Pla

In [26]:
w = net[0].weight
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias
print('b的估计误差：', true_b - b)

w的估计误差： Tensor(shape=[2], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [ 0.00107181, -0.00071168])
b的估计误差： Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [0.00008011])
